# Maximum of the Laplacian

This method yielded an accuracy of **63.58%**.

- The result of the *Laplacian Operator* is converted to an absolute scale (0-255)
- The max of the values is taken for each image
- The threshold is set at 215:
    - Values lower than 215, classify the image as out-of-focus or blurry
    - Greater than 215 is classified as non-blurred

----

### Imports

In [1]:
import cv2

In [2]:
import os
import pickle
import numpy as np
import pandas as pd

In [3]:
#from keras.models import Sequential
#from keras.layers.convolutional import Convolution2D
#from keras.layers.pooling import MaxPooling2D
#from keras.layers.core import Flatten, Dense, Activation, Dropout
from keras.preprocessing import image
#from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [4]:
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [5]:
#from matplotlib import pyplot as plt

In [6]:
from scipy import stats

---

### Algorithm

In [7]:
input_size = (192, 192)
threshold = 215

In [8]:
def variance_of_laplacian(image):
    """
    Compute the Laplacian of the image and then return the focus measure,
    which is simply the variance of the Laplacian
    """
    return cv2.Laplacian(image, cv2.CV_64F).var()

In [9]:
X_test = []
y_test = []
y_pred = []
val_blur = []
val_nblur = []

*Digital-Blur-Set* and *Natural-Blur-Set*:

In [10]:
dgbset = pd.read_excel('../CERTH_ImageBlurDataset/EvaluationSet/DigitalBlurSet.xlsx')
nbset = pd.read_excel('../CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet.xlsx')

In [11]:
dgbset['MyDigital Blur'] = dgbset['MyDigital Blur'].apply(lambda x: x.strip())
dgbset = dgbset.rename(index=str, columns={"Unnamed: 1": "Blur Label"})

In [12]:
nbset['Image Name'] = nbset['Image Name'].apply(lambda x: x.strip())

Run for the artificially-generated images:

In [13]:
folderpath = '../CERTH_ImageBlurDataset/EvaluationSet/DigitalBlurSet/'

In [14]:
for filename in os.listdir(folderpath):
    imagepath = os.path.join(folderpath, filename)
    img = image.load_img(imagepath, target_size=input_size)
    X_test.append(np.asarray(img))

    blur = dgbset[dgbset['MyDigital Blur'] == filename].iloc[0]['Blur Label']
    gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
    maxval = np.max(cv2.convertScaleAbs(cv2.Laplacian(gray, 3)))

    if maxval < threshold:
        y_pred.append(1)
    else:
        y_pred.append(0)

    if blur == 1:
        y_test.append(1)
        val_blur.append(maxval)
    else:
        y_test.append(0)
        val_nblur.append(maxval)

print("Digitally Blurred Evaluated...")

Digitally Blurred Evaluated...


Run for the natural images:

In [15]:
folderpath = '../CERTH_ImageBlurDataset/EvaluationSet/NaturalBlurSet/'

In [16]:
for filename in os.listdir(folderpath):
    imagepath = os.path.join(folderpath, filename)
    img = image.load_img(imagepath, target_size=input_size)
    X_test.append(np.asarray(img))
    
    blur = nbset[nbset['Image Name'] == filename.split('.')[0]].iloc[0]['Blur Label']
    gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
    maxval = np.max(cv2.convertScaleAbs(cv2.Laplacian(gray, 3)))
    
    if maxval < threshold:
        y_pred.append(1)
    else:
        y_pred.append(0)
    
    if blur == 1:
        y_test.append(1)
        val_blur.append(maxval)
    else:
        y_test.append(0)
        val_nblur.append(maxval)

In [17]:
print("Naturally Blurred Evaluated...")

Naturally Blurred Evaluated...


Compute accuracy:

In [18]:
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: {:.2f}%".format(accuracy * 100))

accuracy: 63.58%


More stats:

In [19]:
print(np.percentile(val_blur,60))
print(np.percentile(val_nblur,15))

255.0
255.0
DescribeResult(nobs=861, minmax=(17, 255), mean=201.98606271777004, variance=5369.678875293736, skewness=-1.0729189514344002, kurtosis=-0.3269821207133945)
DescribeResult(nobs=619, minmax=(188, 255), mean=254.8675282714055, variance=7.610254560283579, skewness=-23.316063879349564, kurtosis=557.8368200149888)
